In [1]:
import os
import time
import numpy as np
import pandas as pd
from IPython.display import clear_output 

In [2]:
new_train_correlations = False ### will run for 10 hours for data from 1970 for all symbols, will delete old data
calc_predict_correlations =False ## will create new correlations for the 'periods' after deleting old data
update_train_correlations = True ## update train correlations for the missing dates if all tickers exist

period = 30 # number of days to calculate predict correlations
data_file = '../../prices/adjclose.csv' # correlations on adjclose
train_save_path = '../processed/'
predict_save_path = '../data/predict/'

In [3]:
with open('../../sp500tickers.txt','r') as f:
    symbols=[line.rstrip('\n') for line in f]

In [4]:
df=pd.read_csv(data_file, index_col=0, parse_dates=True)[:]
df.tail()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,^GSPC,^IXIC,^NDX,^SOX,^TNX,^VIX,^VVIX,^VXN,^VXO,^VXV
2021-07-28,151.46,21.63,212.25,144.98,118.55,121.34,323.80,120.52,316.31,620.92,...,4400.64,14762.58,15018.10,3272.45,1.26,18.31,117.08,20.96,15.93,NaN
2021-07-29,152.67,21.17,213.39,145.64,118.87,122.71,324.81,121.09,318.35,621.70,...,4419.15,14778.26,15048.36,3333.24,1.27,17.70,114.04,20.55,15.41,NaN
2021-07-30,153.23,20.38,212.06,145.86,116.30,122.17,327.14,120.98,317.68,621.63,...,4395.26,14672.68,14959.90,3356.53,1.24,18.24,116.18,20.81,16.21,NaN
2021-08-03,153.86,19.95,215.40,147.36,116.25,123.71,326.52,122.12,318.52,621.28,...,4423.15,14761.29,15061.42,3396.66,1.18,18.04,117.55,20.52,14.91,NaN
2021-08-04,154.18,19.45,209.73,146.95,115.07,125.51,331.20,121.62,316.80,625.68,...,4402.66,14780.53,15083.39,3436.90,1.18,17.94,116.02,20.66,15.71,NaN


In [5]:
def train_correlations(df):
    for i in df.index[::-1]:
        date=pd.to_datetime(i).date()
        time.sleep(.1); clear_output()
        print(date)
        corr=df[pd.to_datetime(df.index[0]).date():i].corr()
        for symbol in symbols:
            df1 = corr.drop(symbol, axis=1)
            df1=df1.loc[[symbol],:]
            df1.rename(index={symbol:date},inplace=True)

            if not os.path.exists(train_save_path+symbol+'.csv'):
                df1=df1.add_prefix('corr'+symbol+'_')
                df1.to_csv(train_save_path+symbol+'.csv')
            else:
                df1.to_csv(train_save_path+symbol+'.csv', mode='a', header = False)

            del df1
        del corr    

In [6]:
def predict_correlations(df, period):
    for i in df.index[:-period:-1]:
        date=pd.to_datetime(i).date()
        time.sleep(.1); clear_output()
        print(date)
        corr=df[pd.to_datetime(df.index[0]).date():i].corr()
        for symbol in symbols:
            df1 = corr.drop(symbol, axis=1)
            df1=df1.loc[[symbol],:]
            df1.rename(index={symbol:date},inplace=True)

            if not os.path.exists(predict_save_path+symbol+'.csv'):
                df1=df1.add_prefix('corr'+symbol+'_')
                df1.to_csv(predict_save_path+symbol+'.csv')
            else:
                df1.to_csv(predict_save_path+symbol+'.csv', mode='a', header = False)

            del df1
        del corr    

In [7]:
%%time
if new_train_correlations:# uncommented to prevent deleting data by mistake or generate duplicate data
    for symbol in symbols:
        if os.path.exists(train_save_path+symbol+'.csv'):
            os.remove(train_save_path+symbol+'.csv') 
    train_correlations(df)
    df = pd.read_csv(train_save_path+'^GSPC.csv', index_col=0)
    df=df.sort_index()
    print(df.iloc[-5:,0:4]) # display 5 rows and 4 columns
    del df

Wall time: 0 ns


In [8]:
if calc_predict_correlations:
    for symbol in symbols:
        if os.path.exists(predict_save_path+symbol+'.csv'): 
            os.remove(predict_save_path+symbol+'.csv')  
    predict_correlations(df, period)
    df = pd.read_csv(predict_save_path+'^GSPC.csv', index_col=0)
    df=df.sort_index()
    print(df.iloc[-5:,0:4]) # display 5 rows and 4 columns
    del df

In [9]:
def find_download_dates():
    df_date = []
    for i in df.index[::-1]:
        df_date.append(pd.to_datetime(i).date())
        
    existing_date = []
    symbol = '^GSPC'
    existing = pd.read_csv(train_save_path+symbol+'.csv', index_col=0)
    for j in existing.index[::]:
        existing_date.append(pd.to_datetime(j).date())
    
    download_date = []
    for k in df_date:
        if k in existing_date:
            break
        else:
            download_date.append(k)
    del df_date, existing_date, existing
    return download_date 

In [10]:
%%time
if update_train_correlations:
    download_date = find_download_dates()
    for i in download_date:
        date=pd.to_datetime(i).date()
        time.sleep(.1); clear_output()
        print(date)
        corr=df[pd.to_datetime(df.index[0]).date():i].corr()
        for symbol in symbols[:]:
            df1 = corr.drop(symbol, axis=1)
            df1=df1.loc[[symbol],:]
            df1.rename(index={symbol:date},inplace=True)
            df1.to_csv(train_save_path+symbol+'.csv', mode='a', header = False)
    # sample view of resulting data
    df=pd.read_csv(train_save_path+'^GSPC'+'.csv', parse_dates=True, index_col=0)
    df=df.sort_index()
    print(df.iloc[-5:,0:4]) # display 5 rows and 4 columns
    del df

2021-08-04
            corr^GSPC_A  corr^GSPC_AAL  corr^GSPC_AAP  corr^GSPC_AAPL
2021-08-02     0.960224       0.367696       0.864662        0.852223
2021-08-03     0.960222       0.367666       0.864667        0.852222
2021-08-03     0.960229       0.367674       0.864670        0.852224
2021-08-03     0.960229       0.367674       0.864670        0.852224
2021-08-04     0.960271       0.366949       0.864795        0.852372
Wall time: 10.4 s
